In [1]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_tr_1 = pd.read_csv("C:/Users/User/github/data/TFTLCD/ph1_bytime.csv", encoding='cp949')
df_ts_1 = pd.read_csv("C:/Users/User/github/data/TFTLCD/ph2_in.csv", encoding='cp949')
df_ts_2 = pd.read_csv("C:/Users/User/github/data/TFTLCD/ph2_out.csv", encoding='cp949')

df_tr = df_tr_1.replace('Normal', 0)
df_ts_1 = df_ts_1.replace('정상', 0)
df_ts_1 = df_ts_1.replace('이상', 1)
df_ts_2 = df_ts_2.replace('정상', 0)
df_ts_2 = df_ts_2.replace('이상', 1)

df_tr_1 = df_tr_1.rename(columns={'추가정보_정상.이상여부':'Normal/Abnormal'})
df_ts_1 = df_ts_1.rename(columns={'추가정보_정상.이상여부':'Normal/Abnormal'})
df_ts_2 = df_ts_2.rename(columns={'추가정보_정상.이상여부':'Normal/Abnormal'})

df_tr = df_tr_1.iloc[:, 7:49] # train
df_ts = pd.concat([df_ts_1.iloc[:,7:49], df_ts_2.iloc[:,7:49]])
df_tr_x = df_tr.iloc[:,:-1]
df_ts_x = df_ts.iloc[:,:-1]

df_tr_x = df_tr_x.reset_index(drop = True)
#df_tr_x = np.array(df_tr_x)
df_ts_x = df_ts_x.reset_index(drop = True)
#df_ts_x = np.array(df_ts_x)

In [3]:
from sklearn.mixture import GaussianMixture

# GMM: n_components = 모델의 총 수
gm = GaussianMixture(n_components=5, covariance_type = 'full',n_init=10, random_state=42)
gm.fit(df_tr_x)

GaussianMixture(n_components=5, n_init=10, random_state=42)

In [28]:
q = pd.DataFrame(gm.fit_predict(df_tr_x))

In [75]:
group = gm.predict(df_ts_x)
group = pd.DataFrame(group)

In [30]:
pd.concat([df_tr,group], axis = 1)

,MP_ALLSTEP.INNER_TEMP.Mean,MP_ALLSTEP.INNER_TEMP.Stdev,MP_ALLSTEP.OUTER_TEMP.Mean,MP_ALLSTEP.OUTER_TEMP.Stdev,MP_ALLSTEP.WALLHTR_TEMP1.Mean,MP_ALLSTEP.WALLHTR_TEMP1.Stdev,MP_STEP1_First10sec.PC_PRESSURE.Area,MP_STEP1_First10sec.PC_PRESSURE.Min,MP_STEP3.MATCHER_VDC.Mean,MP_STEP3.MATCHER_VDC.Stdev,...,MP_STEP4.RFGEN_FORWARD.Mean,MP_STEP4.RFGEN_FORWARD.Stdev,MP_STEP4.SIH4_FLOW.Mean,MP_STEP4.SIH4_FLOW.Stdev,MP_STEP4_No_Shift.RFGEN_REFLECT.Max,MP_STEP5_to_Last_PWR_Lift.RFGEN_REFLECT.Max,MP_STEP4.EI2.Max,MP_STEP4.EI3.Max,MP_STEP4.EI4.Max,0
0,339.4905,0.500300,359.1997,0.400100,135.0000,0.0000,0.099300,0.003,35.5000,0.505500,...,22744.9012,2.813200,6784.9612,3.471800,224.8672,910.944300,101.18,100.73,100.73,0
1,336.8500,0.635761,358.4300,0.495240,131.0000,0.0000,0.083875,0.010,26.8600,0.350605,...,22723.3000,2.758552,6786.6800,2.423288,272.2300,669.474455,99.95,100.07,100.73,3
2,339.6100,0.488659,359.5100,0.500244,132.0000,0.0000,0.108875,0.010,32.3000,0.461522,...,22970.2700,2.832701,6804.1100,2.387673,218.0300,399.194287,98.88,98.67,98.02,1
3,339.6200,0.484780,359.4600,0.499061,132.0000,0.0000,0.108375,0.010,33.0000,0.000000,...,22969.8800,2.556473,6802.9200,2.743171,241.4700,421.412440,98.86,98.68,98.00,0
4,337.9200,0.669948,358.8000,0.399836,132.0000,0.0000,0.108250,0.010,36.3600,0.486607,...,22970.5500,2.751402,6803.8300,2.345653,227.0600,163.095892,97.96,97.90,97.58,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,339.5889,0.492400,359.5938,0.491500,135.0000,0.0000,0.074900,0.003,41.2000,0.625200,...,22743.8611,2.709000,6785.1308,2.979600,204.1140,571.568100,101.04,102.23,103.63,2
1996,339.2407,0.504200,359.0048,0.069500,135.0000,0.0000,0.076300,0.004,25.5556,0.692700,...,22744.0548,2.473000,6784.1694,2.636100,183.3608,884.087200,100.97,103.47,106.01,3
1997,339.4857,0.500200,359.4027,0.490800,136.0000,0.0000,0.071500,0.003,38.1778,0.386600,...,22744.6248,3.398000,6784.9828,2.448500,165.2933,863.089800,100.98,102.70,104.75,3
1998,339.4253,0.494800,359.3864,0.487300,136.1315,0.3382,0.066800,0.003,40.7333,0.447200,...,22743.9041,2.858600,6783.9272,1.226200,216.5660,164.805000,100.67,102.35,104.17,0


In [32]:
import sys
sys.path.append('C:/Users/User/github/Function/python/')
from hotellings_tsquare import *

In [34]:
hotellings_tsquare(df_tr, df_tr)

array([44.79476186, 51.89807271, 29.47302337, ..., 46.72009599,
       37.20408378, 52.24665673])

In [59]:
import numpy as np

def hotellings_tsquare(trdat, tedat):
    '''
    Hotellings T-square : 공분산을 반영한 유클리디안 거리 (마할라노비스 거리)
    
    @param trdat : train data
    @param tedat : test data
    '''
    obs = trdat.shape[0]
    dim = trdat.shape[1]
    
    mu = trdat.apply(np.mean, axis=0) # 열단위 평균계산
    sinv = np.linalg.pinv(trdat.cov()) # 공분산 역행렬

    dte = (tedat-mu).values 
    
    Tsq_mat = np.zeros(tedat.shape[0])
    for i in range(tedat.shape[0]):
        Tsq_mat[i] = dte[i,:] @ sinv @ dte[i,:].transpose() # t-square distance
        
    return Tsq_mat

def bootstrap_limit(stat, alpha=0.05, bootstrap=100):
    '''
    Bootstrap sampling을 활용한 Control Limit 산출 기법
    
    @param stat : 통계량
    @param alpha : Control Limit을 정하기 위한 유의수준
    @param : bootstrap : 샘플링 횟수
    '''
    alpha = (alpha/2)*100
    alpha = 100 - alpha
    samsize = max(10000, len(stat))
    limit = np.mean(list(map(lambda x:np.percentile(np.random.choice(stat,samsize,replace=True),alpha), range(0,bootstrap))))
    # stat에서 samsize만큼 선택 후 75%분위수(alpha에따라 변경) 뽑기 이후 bootstrap만큼 반복 후 리스트로 만들고 평균 계산
    return limit

In [40]:
from sklearn import datasets
iris = datasets.load_iris()

In [68]:
trdat = pd.DataFrame(iris.data[:100,:])
tedat = pd.DataFrame(iris.data[100:150,:])

In [72]:
obs = trdat.shape[0]
dim = trdat.shape[1]

mu = trdat.apply(np.mean, axis=0) # 열단위 평균계산
sinv = np.linalg.pinv(trdat.cov()) # 공분산 역행렬

dte = (tedat-mu).values 

Tsq_mat = np.zeros(tedat.shape[0])
for i in range(tedat.shape[0]):
    Tsq_mat[i] = dte[i,:] @ sinv @ dte[i,:].transpose() # t-square distance   


In [96]:
gm = GaussianMixture(n_components=5, covariance_type = 'full',n_init=10, random_state=42)
gm.fit(trdat)

group = gm.predict(trdat)
group = pd.DataFrame(group)
group.columns = ['a']

q = pd.DataFrame(gm.fit_predict(trdat))
w = pd.concat([trdat,group], axis = 1)



In [97]:
w[['a']]

,a
0,1
1,1
2,1
3,1
4,1
...,...
95,3
96,3
97,0
98,2


In [98]:
w[['a']] == 1

,a
0,True
1,True
2,True
3,True
4,True
...,...
95,False
96,False
97,False
98,False
